In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [ ]:

import streamlit as st
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras

# Load the trained model
model = joblib.load('model.pkl')

# Load the LabelEncoder (you'll need to save it during training)
# Replace 'label_encoder.pkl' with the actual filename you used to save it
# label_encoder = joblib.load('label_encoder.pkl')  # Use joblib or pickle to load it


# Function to preprocess input data
def preprocess_input(country_name, language_name):
    # Load the dataset to access unique values (necessary for Label Encoding)
    data = pd.read_csv('/content/books_project.csv')
    data.dropna(axis=0, inplace=True)

    input_data = pd.DataFrame({'country_name': [country_name], 'language_name': [language_name]})

    # Define the categorical columns (same as in your training code)
    categorical_cols = ['language_name', 'country_name']
    label_encoder = LabelEncoder()


    for col in categorical_cols:
        # Handle unseen values gracefully.
        # If a new country_name or language_name is given, handle the 'unknown' class
        unique_values = data[col].unique()
        if input_data[col].iloc[0] not in unique_values:
            input_data[col] = 'unknown'
        input_data[col] = label_encoder.fit_transform(input_data[col])
        #print(f"Encoding: {input_data[col].iloc[0]}")

    return input_data

# Streamlit app
st.title("Book Title Predictor")

# Get user input
country_name = st.text_input("Enter Country Name")
language_name = st.text_input("Enter Language Name")

if st.button("Predict"):
    try:
        # Preprocess the input
        input_df = preprocess_input(country_name, language_name)

        # Make the prediction
        prediction = model.predict(input_df)

        # Get the predicted class (replace with your actual class mapping)
        predicted_class = np.argmax(prediction)

        # Reverse Label Encoding to get the original country name back.
        data = pd.read_csv('/content/books_project.csv')
        data.dropna(axis=0, inplace=True)
        label_encoder = LabelEncoder()
        y = data['country_name']  # Target variable
        y = label_encoder.fit_transform(y)
        predicted_country_name = label_encoder.inverse_transform([predicted_class])[0]

        st.write(f"Predicted Country Name: {predicted_country_name}")
    except Exception as e:
        st.error(f"An error occurred: {e}")